In [2]:
# This code analyze licking parameters from mat files generated by "Licking_Optogenetics_LEDCue.m" and save them in a csv file.
# The parameters analyzed by this code are: "Licks/Bout", "MultiLicks/Bout", "Bout Duration", "Frequency"


import csv
import os
import numpy as np
from scipy.io import loadmat

# Define column names
field_names = ["Name", "Licks/Bout", "MultiLicks/Bout", "Bout Duration", "Frequency"]

# change a working directory
os.chdir('/Users/jun/Documents/Work/Project/Intralingual parasympathetic ganglion neurons/Behavior/ILPG_silencing_#2/')

# get a list of mat files in the directory
mat_files = [f for f in os.listdir() if f.endswith('.mat')]

# Process each .mat file
for mat_file in mat_files:
    # Load the .mat file
    mat = loadmat(mat_file)
    session_data = mat['SessionData']
    n_trials = session_data['nTrials'][0, 0][0, 0]

    raw_events = session_data['RawEvents'][0, 0]['Trial'][0, 0]

    reward = {
        'BNCHigh': [np.nan] * n_trials,
        'BNCLow': [np.nan] * n_trials
    }

    for i in range(n_trials):
        trial = raw_events[0, i]['Events'][0, 0]

        if 'BNC1High' not in trial.dtype.names or 'BNC1Low' not in trial.dtype.names:
            reward['BNCHigh'][i]= np.nan
            reward['BNCLow'][i] = np.nan
        else:
            reward['BNCHigh'][i] = trial['BNC1High'][0, 0]
            reward['BNCLow'][i] = trial['BNC1Low'][0, 0]

    # Remove empty cells
    reward["BNCHigh"] = [x for x in reward["BNCHigh"] if not np.isnan(x).all()]
    reward["BNCLow"] = [x for x in reward["BNCLow"] if not np.isnan(x).all()]

    # Reward delivery time
    trials = session_data["RawEvents"][0, 0]["Trial"]

    # Extract "States" for each trial
    states_list = [trial.item()[0, 0]["States"][0, 0] for trial in np.nditer(trials, flags=['refs_ok'])]

    # Extract "Drinking" states for each trial
    drinking_list = [states['Drinking'][0] for states in states_list]

    # concatenate trials

    licks = [None] * len(reward["BNCHigh"])

    for i in range(1, len(reward["BNCHigh"]) - 1):
        if isinstance(reward["BNCHigh"][i], np.ndarray) and not np.isnan(reward["BNCHigh"][i]).any():
            licks[0] = reward["BNCHigh"][0]
            licks[i] = reward["BNCHigh"][i] + 20 * i

    licks = [x for x in licks if x is not None]  # concatenate non-None elements in lick

    # collapse cells: list-to-array conversion
    licks = np.concatenate([arr.ravel() for arr in licks])

    # find lick bout

    bouts = []
    currentbout = []

    # Iterate over the elements of the licks list
    for i in range(len(licks)):
        # Add the current data point to the current aggregate
        currentbout.append(licks[i])

        # If the gap between the current data point and the        next one is greater than 0.25 (4Hz),
        # store the current aggregate and start a new one 
        if i < len(licks) - 1 and licks[i+1] - licks[i] > 0.25:
            bouts.append(currentbout)
            currentbout = []

    # Store the last aggregate
    if currentbout:
        bouts.append(currentbout)

    # count number of licks in one lick bout
    num_licks = [len(bout) for bout in bouts]
    mean_num_licksperbout = np.mean(num_licks)
    print(mean_num_licksperbout)

    # exclude single licks
    multi_licks = [nlicks for nlicks in num_licks if nlicks != 1]
    mean_num_multi_licks = np.mean(multi_licks)
    print(mean_num_multi_licks)

    # mean lick bout duration
    # Find bouts with more than one data point
    bouts = [b for b in bouts if len(b) > 1]

    # Calculate differences for each bout
    differences = [b[-1] - b[0] for b in bouts]

    # Calculate the mean bout duration
    mean_bout_duration = sum(differences) / len(differences)
    print(mean_bout_duration)

    # Frequency
    mean_frequencies = []
    # bout_diff_store = [] # this is for checking a result. Not necessary for frequency calculation 
    for bout in bouts:
        if len(bout) > 5:
            bout_diff = np.diff(bout)
            # create a mask to select elements larger than or equal to 0.08. Max freqency allowed is 12Hz
            mask = bout_diff >= 0.08
            bout_diff = bout_diff[mask]
            # bout_diff_store.append(bout_diff) # this is for checking a result
            bout_freq = 1 / bout_diff
            mean_frequency = np.mean(bout_freq)
            mean_frequencies.append(mean_frequency)

    print(mean_frequencies)
    print(np.mean(mean_frequencies))

    # At the end of the processing, append the results to the CSV file
    # Check if file exists
    file_exists = os.path.isfile("results.csv")

    # Open the CSV file in append mode
    with open("results.csv", "a", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write the column names only if the file is empty
        if not file_exists:
            csv_writer.writerow(field_names)

        # Write the data row
        csv_writer.writerow([mat_file, mean_num_licksperbout, mean_num_multi_licks, mean_bout_duration, np.mean(mean_frequencies)])



2.706086956521739
3.651351351351351
0.44527054054053816
[6.025397608592274, 5.941287968896292, 5.8833398924946065, 5.790646722943936, 5.5331862704420125, 6.0687595607614435, 5.727077623393611, 5.3123665586123145, 5.304060400674202, 5.876004230568452, 5.589104333977313, 6.962089389182796, 5.5371474620511005, 5.924787802057372, 7.322175859357806, 6.023657599348047, 5.2296624432701115, 5.933309489161136, 5.80920595024699, 5.331516655286408, 5.837482520163104, 5.940862972032322, 5.657010261619777, 5.6859721830702314, 5.529416648350496, 5.694207598478759, 5.625378770089405, 5.38847417144476, 6.068569764990296, 5.491710057294474, 5.454012904037449, 5.664627814625275, 5.614194778962102, 5.660429395668173, 5.330607020368942, 5.758154790387233, 5.191699007857728, 5.937377447209426, 5.859143184708202, 5.712277981100622, 5.457817745820654, 5.372599316010113, 5.743423105839964]
5.739540308405761
3.650273224043716
4.337155963302752
0.5057612385321087
[5.737257695448536, 5.213804997160424, 5.1923141